In [142]:
import pdfplumber
import pandas as pd

In [136]:
import pdfplumber
from itertools import groupby

pdf_raw_data = []

def extract_table(page, y_tolerance=3):
	"""Extract rows by grouping words by their y position."""
	words = page.extract_words(use_text_flow=True)
	if not words:
		return []

	# Sort words by vertical position
	words.sort(key=lambda w: round(w["top"], 1))
	rows = []
	for _, group in groupby(words, key=lambda w: round(w["top"], y_tolerance)):
		row_words = list(group)
		row_words.sort(key=lambda w: w["x0"])  # left-to-right order
		rows.append(row_words)
	return rows


with pdfplumber.open("glosa_n_19(4to_trim_2020)_biobio.pdf") as pdf:

	for page_number, page in enumerate(pdf.pages, start=1):
		print(f"Processing page {page_number}...")
		
		page_list = []
		table_rows = extract_table(page)

		for row_words in table_rows:
			# Skip empty or useless rows
			if not row_words:
				continue

			first_word = row_words[0]
			indent = int(first_word["x0"])


			page_list.append({
				"content": [w["text"] for w in row_words],  # list of cell values
				"indent": indent,
			})
		
		pdf_raw_data.append(page_list)
		print(f"Extracted {len(table_rows)} raw rows\n")


Processing page 1...
Extracted 64 raw rows

Processing page 2...
Extracted 62 raw rows

Processing page 3...
Extracted 62 raw rows

Processing page 4...
Extracted 63 raw rows

Processing page 5...
Extracted 59 raw rows

Processing page 6...
Extracted 61 raw rows

Processing page 7...
Extracted 59 raw rows

Processing page 8...
Extracted 61 raw rows

Processing page 9...
Extracted 61 raw rows

Processing page 10...
Extracted 63 raw rows

Processing page 11...
Extracted 59 raw rows

Processing page 12...
Extracted 63 raw rows

Processing page 13...
Extracted 61 raw rows

Processing page 14...
Extracted 62 raw rows

Processing page 15...
Extracted 61 raw rows

Processing page 16...
Extracted 62 raw rows

Processing page 17...
Extracted 61 raw rows

Processing page 18...
Extracted 63 raw rows

Processing page 19...
Extracted 62 raw rows

Processing page 20...
Extracted 59 raw rows

Processing page 21...
Extracted 60 raw rows

Processing page 22...
Extracted 61 raw rows

Processing page 23.

In [137]:
for row in pdf_raw_data[0]:
	print(row)

{'content': ['CARABINEROS', 'DE', 'CHILE'], 'indent': 245}
{'content': ['_______________________________________________________________________________________________________________'], 'indent': 54}
{'content': ['3.10.', 'REGIÓN', 'DEL', 'BIOBÍO'], 'indent': 75}
{'content': ['CANTIDAD', 'DE', 'CASOS', 'CANTIDAD', 'DE', 'DETENIDOS'], 'indent': 286}
{'content': ['Delitos', 'de', 'Mayor', 'Connotación', 'Social', '(D.M.C.S.)'], 'indent': 73}
{'content': ['Comunas', 'con', 'Plan', 'Cuadrante', 'de', 'Seguridad', 'Preventiva', '(P.C.S.P.)'], 'indent': 56}
{'content': ['1°', '2°', '3°', '4°', '1°', '2°', '3°', '4°'], 'indent': 253}
{'content': ['Anual', 'Anual'], 'indent': 374}
{'content': ['Trimestre', 'Trimestre', 'Trimestre', 'Trimestre', 'Trimestre', 'Trimestre', 'Trimestre', 'Trimestre'], 'indent': 243}
{'content': ['1°,', '2°,', '3°', 'y', '4°', 'Trimestres', 'año', '2020'], 'indent': 93}
{'content': ['REGIÓN', 'DEL', 'BIOBÍO', '7.628', '6.827', '5.389', '5.397', '25.241', '2.205', 

In [138]:
def classifyRow(row):
	indent = row["indent"]

	match indent:
		case 66:
			row_type = "city"
		case 76:
			row_type = "area"
		case 82:
			row_type = "general_crime"
		case 87:
			row_type = "robbery_type"
		case 92:
			row_type = "specific_robbery"
		case _:
			row_type = "other"

	row.update({"row_type": row_type})

def deleteFillerRows(page):
	clean_rows = []

	for row in page:
		if(row["row_type"] != "other"):
			clean_rows.append(row)

	return clean_rows

for i, page in enumerate(pdf_raw_data):
	for row in page:
		classifyRow(row)

	pdf_raw_data[i] = deleteFillerRows(page)

for row in pdf_raw_data[0]:
	print(row["row_type"], row["content"])

city ['ARAUCO', '167', '156', '107', '70', '500', '39', '50', '37', '32', '158']
area ['Cuadrante', '01', '85', '58', '51', '19', '213', '20', '21', '14', '6', '61']
general_crime ['Robos', '42', '33', '26', '11', '112', '4', '8', '4', '2', '18']
robbery_type ['Robos', 'Violentos', '5', '8', '6', '5', '24', '0', '3', '2', '1', '6']
specific_robbery ['Robo', 'con', 'violencia', '0', '1', '3', '2', '6', '0', '0', '2', '1', '3']
specific_robbery ['Robo', 'con', 'intimidación', '3', '4', '2', '0', '9', '0', '3', '0', '0', '3']
specific_robbery ['Robo', 'por', 'sorpresa', '2', '3', '1', '3', '9', '0', '0', '0', '0', '0']
robbery_type ['Robos', 'con', 'Fuerza', '37', '25', '20', '6', '88', '4', '5', '2', '1', '12']
specific_robbery ['Robo', 'de', 'Vehículo', 'Motorizado', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']
specific_robbery ['Robo', 'de', 'Accesorio', 'de', 'Vehículo', '1', '4', '6', '1', '12', '0', '4', '0', '0', '4']
specific_robbery ['Robo', 'en', 'Lugar', 'Habitado', '13', 

In [139]:
def convertDataType(row):
	row_content = row["content"]
	modified_row = []
	aux_text = ""
	end_text = False

	for value in row_content:
		# Deletes dots for numeric values
		value = value.replace(".", "")

		if(not str.isdigit(value)):
			aux_text += value + " "
		else:
			if(not end_text):
				aux_text = aux_text.strip()
				modified_row.append(aux_text)
				end_text = True
			modified_row.append(int(value))

	if(row["row_type"] == "area" and len(modified_row) == 12):
		aux_row = []
		aux_row.append(modified_row[0] + " " + str(modified_row[1]))
		modified_row = aux_row + modified_row[2:]

	return modified_row

for i, page in enumerate(pdf_raw_data):
	for j, row in enumerate(page):
		pdf_raw_data[i][j]["content"] = convertDataType(row)

for row in pdf_raw_data[0]:
	print(row["row_type"], row["content"])

city ['ARAUCO', 167, 156, 107, 70, 500, 39, 50, 37, 32, 158]
area ['Cuadrante 1', 85, 58, 51, 19, 213, 20, 21, 14, 6, 61]
general_crime ['Robos', 42, 33, 26, 11, 112, 4, 8, 4, 2, 18]
robbery_type ['Robos Violentos', 5, 8, 6, 5, 24, 0, 3, 2, 1, 6]
specific_robbery ['Robo con violencia', 0, 1, 3, 2, 6, 0, 0, 2, 1, 3]
specific_robbery ['Robo con intimidación', 3, 4, 2, 0, 9, 0, 3, 0, 0, 3]
specific_robbery ['Robo por sorpresa', 2, 3, 1, 3, 9, 0, 0, 0, 0, 0]
robbery_type ['Robos con Fuerza', 37, 25, 20, 6, 88, 4, 5, 2, 1, 12]
specific_robbery ['Robo de Vehículo Motorizado', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
specific_robbery ['Robo de Accesorio de Vehículo', 1, 4, 6, 1, 12, 0, 4, 0, 0, 4]
specific_robbery ['Robo en Lugar Habitado', 13, 9, 12, 3, 37, 0, 1, 2, 0, 3]
specific_robbery ['Robo en Lugar No Habitado', 22, 12, 2, 2, 38, 4, 0, 0, 1, 5]
specific_robbery ['Otros Robos con Fuerza', 1, 0, 0, 0, 1, 0, 0, 0, 0, 0]
general_crime ['Homicidios', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
general_crime ['Viola

In [ ]:
ordered_data = []

city = ""
area = ""

for page in pdf_raw_data:
  aux_area_data = {}

  for row in page:
    if(row["row_type"] == "city"):
      city = row["content"][0]

    elif(row["row_type"] == "area"):
      area = row["content"][0]

    else:
      row_title = row["content"][0]

      # Saves crime name and its anual value
      aux_area_data.update({row_title: row["content"][5]})

      # Last crime name for every area marks the start of another dictionary
      if(row_title == "Hurtos"):
        aux_area_data.update({"City": city, "Area": area})
        ordered_data.append(aux_area_data)
        aux_area_data = {}

for element in ordered_data:
  print(element)

{'Robos': 112, 'Robos Violentos': 24, 'Robo con violencia': 6, 'Robo con intimidación': 9, 'Robo por sorpresa': 9, 'Robos con Fuerza': 88, 'Robo de Vehículo Motorizado': 0, 'Robo de Accesorio de Vehículo': 12, 'Robo en Lugar Habitado': 37, 'Robo en Lugar No Habitado': 38, 'Otros Robos con Fuerza': 1, 'Homicidios': 0, 'Violaciones': 0, 'Lesiones': 22, 'Hurtos': 79, 'City': 'ARAUCO', 'Area': 'Cuadrante 1'}
{'Robos': 77, 'Robos Violentos': 5, 'Robo con violencia': 0, 'Robo con intimidación': 4, 'Robo por sorpresa': 1, 'Robos con Fuerza': 72, 'Robo de Vehículo Motorizado': 0, 'Robo de Accesorio de Vehículo': 6, 'Robo en Lugar Habitado': 36, 'Robo en Lugar No Habitado': 27, 'Otros Robos con Fuerza': 3, 'Homicidios': 0, 'Violaciones': 2, 'Lesiones': 27, 'Hurtos': 39, 'City': 'ARAUCO', 'Area': 'Cuadrante 2'}
{'Robos': 22, 'Robos Violentos': 4, 'Robo con violencia': 2, 'Robo con intimidación': 1, 'Robo por sorpresa': 1, 'Robos con Fuerza': 18, 'Robo de Vehículo Motorizado': 0, 'Robo de Accesor

In [145]:
df = pd.DataFrame(ordered_data)

# Change column order
cols = df.columns.tolist()
new_order = cols[-2:] + cols[:-2]
df = df[new_order]

df

,City,Area,Robos,Robos Violentos,Robo con violencia,Robo con intimidación,Robo por sorpresa,Robos con Fuerza,Robo de Vehículo Motorizado,Robo de Accesorio de Vehículo,Robo en Lugar Habitado,Robo en Lugar No Habitado,Otros Robos con Fuerza,Homicidios,Violaciones,Lesiones,Hurtos
0,ARAUCO,Cuadrante 1,112,24,6,9,9,88,0,12,37,38,1,0,0,22,79
1,ARAUCO,Cuadrante 2,77,5,0,4,1,72,0,6,36,27,3,0,2,27,39
2,ARAUCO,Cuadrante 3,22,4,2,1,1,18,0,0,7,11,0,0,1,32,22
3,ARAUCO,Cuadrante 4,27,3,0,3,0,24,2,0,3,19,0,0,2,18,18
4,CABRERO,Cuadrante 1,75,10,2,5,3,65,6,29,4,25,1,0,0,17,43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,TOMÉ,Cuadrante 1,117,33,11,16,6,84,22,11,18,29,4,0,1,88,154
71,TOMÉ,Cuadrante 2,59,14,8,3,3,45,6,2,17,16,4,0,2,46,34
72,YUMBEL,Cuadrante 1,53,6,2,2,2,47,3,6,20,14,4,0,0,21,42
73,YUMBEL,Cuadrante 2,49,2,0,0,2,47,0,2,27,16,2,0,0,12,13
